These scripts scrape data from the Utah Water Rights website, saves the data to text files, then parses the text files into a MySQL database.

In [1]:
%matplotlib inline
import pandas as pd
import urllib2
from urllib2 import urlopen
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import matplotlib.ticker as tick
import scipy.stats as sp
import statsmodels.api as sm
from pandas.stats.api import ols
from datetime import datetime
from bs4 import BeautifulSoup
from pylab import rcParams
import platform
rcParams['figure.figsize'] = 15, 10
import re
import os
import sys
import glob
import urllib
import HTMLParser
from cStringIO import StringIO

Choose output routes

In [2]:
route = 'C:/PROJECTS/WR_DATA/'
wellpath = 'C:/PROJECTS/WR_DATA/RawWellogs/'
syspath = 'C:/PROJECTS/WR_DATA/RawSystems/'

Parser and Scraper Functions

In [3]:
def make_soup(url):
    # opens webpage for use in BeautifulSoup    
    html = urlopen(url).read()
    return BeautifulSoup(html, "lxml")

# Scrapers

## Well Log Scraper

Scrapes well Logs from Water Rights website

In [ ]:
# Water Rights win number to begin search
winbegin = 34000
space = 1000
winend = winbegin + space

In [ ]:
while winbegin < 60000:
       
    # opens waterrights webpage by win   
    for i in range(winbegin,winend):
        try:
            win = str(i)
            soup = make_soup('http://waterrights.utah.gov/cgi-bin/docview.exe?Folder=welllog'+str(i))
            souplist = soup.find('a', href=re.compile('^http://waterrights.utah.gov/docSys/v907/.*'))['href']
            soupsite = make_soup(souplist)
            souptext = soupsite.get_text()
            g = path + 'log' + str(win).zfill(5) + '.txt'    
            b = open(g, 'w')
            b.write(souptext.encode('utf-8'))
            b.close()
        except TypeError:        
            pass
    
    winbegin = winend
    winend = winbegin + space

## Water System Use Scraper

From: http://www.waterrights.utah.gov/wateruse/WaterUseList.asp<br/>
Example Pages of Input:<br/>
http://www.waterrights.utah.gov/cgi-bin/wuseview.exe?Modinfo=Indview&SYSTEM_ID=11247<br/>
http://www.waterrights.utah.gov/cgi-bin/wuseview.exe?Modinfo=Mgtview&SYSTEM_ID=11228<br/>

In [ ]:
def systemscraper(winbegin,winfinish,space,prefix,path):
    '''
    Systematically progresses though integer id numbers at the end of Water Rights URL to find system pages, 
    then saves those pages to text files.
    
    INPUT
    -----
    winbegin = integer value to start search
    space = number of integers to search at a time
    winfinish = integer value to end search
    prefix = subset of systems to search (Modinfo= value in URL); can be Pws, Ind, or Mgt
    path = place to store resulting text files
    
    OUTPUT
    ------
    text files in path labeled with corresponding integer values
    '''
    winend = winbegin + space

    while winbegin < winfinish:

        systemnm = []

        # opens waterrights webpage by win   
        for i in range(winbegin,winend):
            try:
                htmlplace = 'http://www.waterrights.utah.gov/cgi-bin/wuseview.exe?Modinfo=' + str(prefix) + 'view&SYSTEM_ID='+str(i)
                soup = make_soup(htmlplace).get_text()
                if "ERROR: Use UNITS undefined" in soup or len(soup) < 1000:
                    pass
                else:
                    systemnm.append(str(i))
                    g = path + str(prefix) + str(i).zfill(6) + '.txt'
                    b = open(g, 'w')
                    b.write(soup.encode('utf-8').strip())   
                    b.close()
            except TypeError:        
                pass

        winbegin = winend
        winend = winbegin + space
    print("Scanned %s to %s"%(prefix,winfinish))

In [ ]:

systems = ['Pws','Ind','Mgt']

for i in systems:
    systemscraper(0,3000,1000,i,syspath)
    systemscraper(10000,13000,1000,i,syspath)

# Parsers

## Well Parsers

This code searches through text captures of Water Rights html well files.

In [4]:
filepath = wellpath + '*.txt'

In [5]:
def raises(exception_types, func, *args, **kw):
    try:
        func(*args, **kw)
    except exception_types:
        return True
    else:
        return False

In [6]:
def tparser(blurb):
    '''
    parses a snippet of text from gettext by removing and replaces extra spaces and return characters
    '''
    blurb = re.sub('\r\n      +', '\n',str(blurb))
    blurb = re.sub('\r\n +','\r\n',blurb)
    blurb = re.sub(',',';',blurb)
    blurb = re.sub(' +',',',blurb)
    blurb = re.sub('\r\n','\n',blurb)
    blurb = re.sub('\n\n','\n',blurb)
    return blurb

def gettext(strttext,endtext,snip):
    '''
    selects a subset of text by searching the text for a beginning string and an ending string
    
    INPUT
    -----
    strttext = string to find that begins text subset
    endtext = string to find that ends the text subset
    snip = text to subset
    
    OUTPUT
    ------
    b = subset of text; returns np.nan if no strttext is found
    '''
    
    b = snip[snip.find(strttext)+len(strttext):snip.find(endtext,snip.find(strttext))].strip()
    if snip.find(strttext) == -1:
        return np.nan
    else:
        return b

### Water Level Parser

In [7]:
filepath = wellpath +'*.txt'

wl = []
for f in glob.glob(filepath): 
    text = open(f).read()    

    # grab section out of each text file for parsing
    
    rr =[]
    wellcon = gettext(' WATER LEVEL DATA:','\r\n\r\n',text)
    #print wellcon
    if wellcon is not np.nan:
        if len(wellcon) > 10:    
            win = str(int(os.path.split(f)[1][3:8])).zfill(5)
            rv = wellcon.split('\n')
            rr = []
            for j in range(2,len(rv)):
                if raises(ValueError, int, rv[j][0:21].strip(' ')[0:2])==False:
                    rv[j] = win + ',' + rv[j][0:21] + ',' + rv[j][30:38].replace(',',';').strip(' ') + ',' + rv[j][38:].strip(' ')
                
                    rr.append(rv[j].replace('\r','').replace('  ',' ').strip(' '))
                else:
                    pass
            wl.append('\n'.join(rr))

levs = '\n'.join(wl)

In [8]:
waterlevels = pd.read_csv(StringIO(levs),names=['WIN','Date','Level','Method'],parse_dates=['Date'])
waterlevels.drop_duplicates(inplace=True)

### Drilling Parser

In [9]:
filepath = wellpath +'*.txt'

br = []
for f in glob.glob(filepath): 
    text = open(f).read()    

    # grab section out of each text file for parsing
    
    rr =[]
    wellcon = gettext(' BOREHOLE INFORMATION:','\r\n\r\n',text)
    #print wellcon
    if wellcon is not np.nan:
        if len(wellcon) > 10:    
            win = str(int(os.path.split(f)[1][3:8])).zfill(5)
            rv = wellcon.split('\n')
            rr = []
            for j in range(2,len(rv)):
                if raises(ValueError, int, rv[j][0:21].strip(' ')[0:2])==False:
                    rv[j] = win + ',' + rv[j][0:17] + ',' + rv[j][17:23] + ',' + rv[j][23:29]+ ',' + rv[j][29:58]+ ',' + rv[j][58:]
                
                    rr.append(rv[j].replace('\r','').replace('  ',' ').strip(' '))
                    #print(rv[j])
                else:
                    pass
            br.append('\n'.join(rr))
   
bore = '\n'.join(br)

In [10]:
borehole = pd.read_csv(StringIO(bore),names=['WIN','From_ft','To_ft','Diameter','Method','Fluid'])
borehole.drop_duplicates(inplace=True)

### Drilling Activity Parser

In [11]:
filepath = wellpath +'*.txt'


rr = []
for f in glob.glob(filepath):
    
    text = open(f).read()    

    # grab section out of each text file for parsing
    
    wellcon = gettext(' DRILLER ACTIVITIES:','\r\n\r\n',text)
    
    #print wellcon
    if wellcon is not np.nan:
        if len(wellcon) > 10:    
            win = str(int(os.path.split(f)[1][3:8])).zfill(5)
            
            act1 = str(gettext('ACTIVITY # 1 ','\r\n\r\n',wellcon))
            actnm1 = str(gettext('ACTIVITY # 1 ','\r\n',wellcon)).strip(' ')
            drllr1 = str(gettext('DRILLER: ','LICENSE #:',act1)).replace(',',' ').strip(' ')
            lic1 =  str(gettext('LICENSE #:','\r\n',act1)).strip(' ')
            strt1 = str(gettext('START DATE: ','COMPLETION DATE: ',act1)).strip(' ')
            comp1 = str(gettext('COMPLETION DATE: ','\r\n',act1)).strip(' ')
            rr.append(win+','+actnm1+','+drllr1+','+lic1+','+strt1+','+comp1)
            #print win+','+actnm1+','+drllr1+','+lic1+','+strt1+','+comp1
            if 'ACTIVITY # 2' in wellcon: 
                act2 = str(gettext('ACTIVITY # 2 ','\r\n\r\n',wellcon))
                actnm2 = str(gettext('ACTIVITY # 2 ','\r\n',wellcon)).strip(' ')
                drllr2 = str(gettext('DRILLER: ','LICENSE #:',act2)).replace(',',' ').strip(' ')
                lic2 =  str(gettext('LICENSE #:','\r\n',act2)).strip(' ')
                strt2 = str(gettext('START DATE: ','COMPLETION DATE: ',act2)).strip(' ')
                comp2 = str(gettext('COMPLETION DATE: ','\r\n',act2)).strip(' ')
                rr.append(win+','+actnm2+','+drllr2+','+lic2+','+strt2+','+comp2)
    
            if 'ACTIVITY # 3' in wellcon:
                act3 = str(gettext('ACTIVITY # 3 ','\r\n\r\n',wellcon))
                actnm3 = str(gettext('ACTIVITY # 3 ','\r\n',wellcon)).strip(' ')
                drllr3 = str(gettext('DRILLER: ','LICENSE #:',act3)).replace(',',' ').strip(' ')
                lic3 =  str(gettext('LICENSE #:','\r\n',act3)).strip(' ')
                strt3 = str(gettext('START DATE: ','COMPLETION DATE: ',act3)).strip(' ')
                comp3 = str(gettext('COMPLETION DATE: ','\r\n',act3)).strip(' ')
                rr.append(win+','+actnm3+','+drllr3+','+lic3+','+strt3+','+comp3)
drill = '\n'.join(rr)

In [12]:
driller = pd.read_csv(StringIO(drill),names=['WIN','activity','driller','license','start','completion'],index_col=False)#,parse_dates=['start','completion'])
driller.drop_duplicates(inplace=True)

### Lithology Parser

In [13]:
filepath = wellpath +'*.txt'

lit = []
for f in glob.glob(filepath): 
    text = open(f).read()    

    # grab section out of each text file for parsing
    
    rr =[]
    litho = gettext(' LITHOLOGY:','\r\n\r\n',text)
    #print wellcon
    if litho is not np.nan:
        if len(litho) > 10:    
            win = str(int(os.path.split(f)[1][3:8])).zfill(5)
            rv = litho.split('\n')
            rr = []
            for j in range(2,len(rv)):
                if j == len(rv):
                    rv[j] = win + ',' + rv[j][0:7] + ',' + rv[j][7:13].strip(' ') + ',' + rv[j][13:95].replace(',',';').strip(' ') + ',' + rv[j][95:108].strip(' ') +','+rv[j][108:].strip(' ').replace(',',';').replace('  ',' ') + ','
                else:
                    if len(rv[j][0:8].strip(' ')) < 1:
                        pass
                    else:
                        rv[j] = win + ',' + rv[j][0:7] + ',' + rv[j][7:13].strip(' ') + ',' + rv[j][13:95].replace(',',';').strip(' ') + ',' + rv[j][95:108].strip(' ') +','+rv[j][108:].strip(' ').replace(',',';').replace('  ',' ') + ','
                        try:
                            if len(rv[j+1][0:8].strip(' ')) < 1:
                                if len(rv[j+1].replace('  ',' ').strip(' '))<350:
                                    rv[j] = rv[j] + rv[j+1].replace(',',';').replace('  ',' ').strip(' ')
                                else:
                                    rv[j] = rv[j] + rv[j+1].replace(',',';').replace('  ',' ').strip(' ')[0:350]
                        except(IndexError):
                            pass
                rr.append(rv[j].replace('\r','').replace('  ',' ').strip(' '))
                
                #print(rr)
            lit.append('\n'.join(rr))

                
   
lith = '\n'.join(lit)

In [14]:
lithlog = pd.read_csv(StringIO(lith),names=['WIN','From_ft','To_ft','Material','Color','Rock Type','Comment'])
lithlog.drop_duplicates(inplace=True)
lithlog.From_ft = pd.to_numeric(lithlog.From_ft, errors='coerce')
lithlog.To_ft = pd.to_numeric(lithlog.To_ft, errors='coerce')
lithlog.dropna(subset=['From_ft','To_ft'],inplace=True)

In [15]:
def lithsort(lith, x):      
    if str(lith).lower() in str(x).lower():
        return 1
    else:
        return 0

def lithsorth(lith, x):
    if 'other' in str(x[0]).lower():
        b = str(x[0]).lower() + ' ' + str(x[1]).upper()
    else:
        b = x[0]
    if str(lith).lower() in str(b).lower():
        return 1
    else:
        return 0

    
    
lithlog['low_perm'] = lithlog['Material'].apply(lambda x: lithsort('LOW-PERMEABILITY', x),1)
lithlog['high_perm'] = lithlog['Material'].apply(lambda x: lithsort('HIGH-PERMEABILITY', x),1)
lithlog['clay'] = lithlog['Material'].apply(lambda x: lithsort('clay', x),1)
lithlog['silt'] = lithlog['Material'].apply(lambda x: lithsort('silt', x),1)
lithlog['sand'] = lithlog['Material'].apply(lambda x: lithsort('sand', x),1)
lithlog['gravel'] = lithlog['Material'].apply(lambda x: lithsort('gravel', x),1)
lithlog['cobbles'] = lithlog['Material'].apply(lambda x: lithsort('cobbles', x),1)
lithlog['boulders'] = lithlog['Material'].apply(lambda x: lithsort('boulders', x),1)
lithlog['hardpan'] = lithlog[['Material','Comment']].apply(lambda x: lithsorth('hardpan', x),1)
lithlog['conglomerate'] = lithlog[['Material','Comment']].apply(lambda x: lithsorth('conglomerate', x),1)
lithlog['bedrock'] = lithlog[['Material','Comment']].apply(lambda x: lithsorth('bedrock', x),1)
lithlog['other'] = lithlog['Material'].apply(lambda x: lithsort('other', x),1)
lithlog['water_bearing'] = lithlog['Material'].apply(lambda x: lithsort('water-bearing', x),1)


def unitassign(x):
    clay = x[0]
    silt = x[1]
    sand = x[2]
    gravel = x[3]
    cobbles = x[4]
    boulders = x[5]
    hardpan = x[6]
    conglomerate = x[7]
    bedrock = x[8]
    other = x[9]
    unitlist = [clay,silt,sand,gravel,cobbles,boulders, hardpan,conglomerate,bedrock,other]
    unitindex = ['clay','silt','sand','gravel','cobbles','boulders', 'hardpan','conglomerate','bedrock','other']
    unitsum = np.sum(unitlist)
    j =str("")
    for i in range(len(unitlist)):
        if unitlist[i] == 1:
            if len(j)==0:
                j = unitindex[i]
            else:
                j = j + "-" + unitindex[i]
    return j    

lithlog.units = lithlog[['clay','silt','sand','gravel','cobbles','boulders', 'hardpan','conglomerate','bedrock','other']].apply(lambda x: unitassign(x),1)

consdict = {'other':'other', 'boulders':'gravel', 'gravel':'gravel', 'sand-gravel-cobbles':'sand-gravel',
            'sand-gravel-cobbles-boulders':'sand-gravel', 'clay-boulders':'clay-gravel', 
            'clay-gravel-boulders':'clay-gravel', 'gravel-conglomerate':'conglomerate', 'cobbles':'gravel',
            'gravel-cobbles':'gravel', 'gravel-boulders':'gravel', 'clay-gravel-cobbles-boulders':'clay-gravel', 
            'gravel-cobbles-boulders':'gravel', 'clay-cobbles-boulders':'clay-gravel', 
            'clay-cobbles':'clay-gravel','clay-sand-gravel-cobbles':'clay-gravel', 
            'clay-hardpan':'hardpan', 'cobbles-boulders':'gravel', 'clay-gravel-cobbles':'clay-gravel', 
            'clay-conglomerate':'conglomerate', 'clay-silt-sand-gravel-conglomerate':'conglomerate', 
            'sand-gravel-boulders':'sand-gravel','sand-boulders':'sand-gravel','clay-silt-gravel':'clay-gravel',
           'clay-silt-sand':'clay-sand','clay-silt':'clay-sand','clay-sand-gravel':'clay-gravel',
           'silt-sand':'sand','clay-silt-gravel-cobbles':'clay-gravel','silt-sand-gravel':'sand-gravel'}

lithlog['unitssimp'] = lithlog.units.apply(lambda x:consdict.get(x,x),1)


def otherassign(x):
    if x[0] == 'other' or x[0]=='':
        if str(x[1]).lower().find('soil') >-1:
            return 'soil'
        elif str(x[1]).lower().find('overburden') >-1:
            return 'soil'
        elif str(x[1]).lower().find('limestone') >-1:
            return 'limestone'
        elif str(x[1]).lower().find('shale') >-1:
            return 'shale'
        elif str(x[1]).lower().find('cemented') >-1:
            return 'conglomerate'
        elif str(x[1]).lower().find('conglomerate') >-1:
            return 'conglomerate'
        else:
            return ''
    elif x[0] == 'bedrock':
        if str(x[1]).lower().find('limestone') >-1:
            return 'limestone'
        elif str(x[1]).lower().find('shale') >-1:
            return 'shale'
        elif str(x[1]).lower().find('cemented') >-1:
            return 'conglomerate'
        elif str(x[1]).lower().find('conglomerate') >-1:
            return 'conglomerate'
        else:
            return 'bedrock'
    else:
        return x[0]

lithlog['unitssimp'] = lithlog[['unitssimp','Comment']].apply(lambda x:otherassign(x),1)

unitnumber = {'soil':0, 'sand-gravel':1, 'clay':2, 'gravel':1, 'sand':4, 'clay-gravel':5,
              'clay-sand':3, 'conglomerate':7, 'hardpan':2, 'bedrock':6, 'limestone':6,
              'sand-gravel-other':1, 'clay-silt-sand-gravel':3, 'clay-silt-other':2,
              'silt':2, 'clay-other':2, 'shale':2}
lithlog['unitnumber'] = lithlog['unitssimp'].apply(lambda x: unitnumber.get(x,8),1)

### Construction Parser

In [16]:
const = []
for f in glob.glob(filepath): 
    text = open(f).read()    

    # grab section out of each text file for parsing

    rev = []

    wellcon = gettext('CASING:','\r\n\r\n',text)

    if wellcon is not np.nan:
        if len(wellcon) > 10:    
            win = str(int(os.path.split(f)[1][3:8])).zfill(5)
            rv = wellcon.split('\n')
            rr = []
            for j in range(2,len(rv)):
                #       WIN           From                    To                                          Material                                    Gage                        Diam                                                          
                rv[j] = win + ',' + rv[j][0:20].replace('+','-') + ',' + rv[j][20:24].strip(' ') + ',' + rv[j][24:45].replace(',',';').strip(' ') + ',' + rv[j][45:57].strip(' ') +','+rv[j][57:].strip(' ').replace('  ',' ')

                rr.append(rv[j].replace('\r','').replace('  ',' ').strip(' '))
                
                #print(rr)
            const.append('\n'.join(rr))



constList = '\n'.join(const)

In [17]:
construction = pd.read_csv(StringIO(constList),names=['WIN','From_ft','To_ft','Material','Gage_in','Diameter_in'])
construction.drop_duplicates(inplace=True)
construction.From_ft = pd.to_numeric(construction.From_ft,errors='coerce')
construction.To_ft = pd.to_numeric(construction.To_ft,errors='coerce')
construction.Diameter_in = pd.to_numeric(construction.Diameter_in,errors='coerce')
construction.dropna(subset=['WIN','From_ft','To_ft'],inplace=True)

### Screen Parser

In [18]:
filepath = wellpath +'*.txt'

srn = []
for f in glob.glob(filepath):    
    text = open(f).read()   

    scrntxt = gettext('SCREENS/PERFORATIONS:','\r\n\r\n',text)
    
    if scrntxt is not np.nan:
        if len(scrntxt) > 10:    

            win = str(int(os.path.split(f)[1][3:8])).zfill(5)
            rv = scrntxt.split('\n')
            rr = []
            for j in range(2,len(rv)):
                #       WIN           From                    To                                          Type                                                Slot                        Diam                                                          
                rv[j] = win + ',' + rv[j][0:18].replace('+','-') + ',' + rv[j][18:25].strip(' ') + ',' + rv[j][25:53].replace(',',';').strip(' ') + ',' + rv[j][53:69].strip(' ') +','+ rv[j][69:97].strip(' ').replace('  ',' ') +','+ rv[j][97:].replace(',',';').strip(' ')

                rr.append(rv[j].replace('\r','').replace('  ',' ').strip(' '))
                
                #print(rr)
            srn.append('\n'.join(rr))
            
scrrn = '\n'.join(srn)

screendf = pd.read_csv(StringIO(scrrn),names=['WIN','From_ft','To_ft','Screen Type',
                                    'Slot_Size_in','Scrn_Diam_in','Perfs'])
screendf.drop_duplicates(inplace=True)

screendf.From_ft = pd.to_numeric(screendf.From_ft,errors='coerce')
screendf.To_ft = pd.to_numeric(screendf.To_ft,errors='coerce')

scrnInt = screendf.groupby('WIN').agg({'From_ft':np.min, 'To_ft':np.min})

In [19]:
scrnInt.reset_index(inplace=True)

### Pumping Test Parser

In [20]:
filepath = wellpath +'*.txt'

pmp = []
for f in glob.glob(filepath): 
    text = open(f).read()    

    # grab section out of each text file for parsing
    
    rr =[]
    welltest = gettext('WELL TESTS:','\r\n\r\n\r\n ',text)
    #print wellcon
    if welltest is not np.nan:
        if len(welltest) > 10:    
            win = str(int(os.path.split(f)[1][3:8])).zfill(5)
            rv = welltest.split('\n')
            rr = []
            for j in range(2,len(rv)):
                #       WIN           DATE                    Test Method                                  Yield                                    Drawdown                        Time pump                                                          
                rv[j] = win + ',' + rv[j][0:22] + ',' + rv[j][22:41].replace(',',';').strip(' ') + ',' + rv[j][41:54].strip(' ') + ',' + rv[j][54:70].strip(' ') +','+rv[j][70:].strip(' ').replace('  ',' ')

                rr.append(rv[j].replace('\r','').replace('  ',' ').strip(' '))
                
                #print(rr)
            pmp.append('\n'.join(rr))

                
   
pump = '\n'.join(pmp)

pumpingtests = pd.read_csv(StringIO(pump), names=['WIN', 'Date', 'Method', 'Yield_cfs', 
                                                  'Drawdown_ft', 'Pump_Dur_hr'])#,parse_dates=['Date'])
pumpingtests.drop_duplicates(inplace=True)
pumpingtests['Yield_cfs'] = pd.to_numeric(pumpingtests.Yield_cfs, errors='coerce')
pumpingtests['Drawdown_ft'] = pd.to_numeric(pumpingtests.Drawdown_ft, errors='coerce')
pumpingtests['Pump_Dur_hr'] = pd.to_numeric(pumpingtests.Pump_Dur_hr, errors='coerce')
pumpingtests.dropna(subset=['Yield_cfs','Drawdown_ft','Pump_Dur_hr'],inplace=True)

In [21]:
pumpT = pd.merge(pumpingtests, construction, on='WIN',how='left')

In [22]:
def getTrans(x,S):
    Q = float(x[0])*86400.0
    d = float(x[1])
    t = float(x[2])/24.0
    if d == 0:
        sc = Q/0.1
    else:
        sc = Q/d
        
    r = float(x[3])/24.0
    if r == 0:
        return np.nan

    else:
        T0 = 100.0
        delt = 100.0
        while abs(delt) > 0.01:
            T = (sc/(4*np.pi))*(np.log((2.25*T0*t)/(r*r*S)))
            delt = T - T0
            T0 = T
        return T
             
S = 0.0002

pumpT['trans'] = pumpT[['Yield_cfs','Drawdown_ft','Pump_Dur_hr','Diameter_in']].apply(lambda x: getTrans(x,S),1)

In [23]:
pumpT.drop(['From_ft','To_ft','Material','Gage_in'],axis=1,inplace=True)

In [24]:
wellTrans = pd.merge(pumpT, scrnInt, on='WIN', how='left')
wellTrans.dropna(subset=['trans'],inplace=True)
wellTrans = wellTrans[(wellTrans.Drawdown_ft > 0)]

## System and Source Parser

In [25]:
pathname = 'D:/PROJECTS/WR_DATA/RawSystems/'

In [26]:
files = glob.glob(pathname +'*.txt')

sourcet, conn, use = {}, {}, {}

indcode, systype, sysnum, link, sysname, city, county, syscat, huc, pwsid, deqcat, numberofsources = [],[],[],[],[],[],[],[],[],[],[],[]
source, system, systemid, pls, sourcetype, sourceuse, win, wrnum, sourcecode,sourceid = [],[],[],[],[],[],[],[],[],[]
system_id = []

for f in range(len(files)): 
    text = open(files[f]).read()
    systemname = gettext('System  Name:','Address:',text)
    sid = gettext('Public Water System ID:','DEQ',text)
    srcind = [m.start() for m in re.finditer('Source Summary', text)]
    
    prefix = os.path.split(files[f])[1][0:3]
    html = 'http://www.waterrights.utah.gov/cgi-bin/wuseview.exe?Modinfo='+ prefix +'view&SYSTEM_ID='
    
    sysnum.append(int(os.path.split(files[f])[1][3:9]))
    linknum = int(os.path.split(files[f])[1][3:9])
    systype.append(prefix)
    link.append(html+str(linknum))
    
    systid = prefix+'-'+str(linknum).zfill(5)
    
    system_id.append(systid)
    sysname.append(gettext('System  Name:','Address:',text))
    city.append(gettext('City:','State:',text))
    county.append(gettext('County:','Primary Use:',text))
    syscat.append(gettext('Primary Use:','Standard',text))
    huc.append(gettext('Hydro Unit Code:','Public',text))
    pwsid.append(gettext('Public Water System ID:','DEQ',text))
    deqcat.append(gettext('DEQ System Category:','\n',text))
    indcode.append(gettext('Standard Industrial Code:','Dual',text))
    
    numberofsources.append(len(srcind))
    
    for i in range(len(srcind)):

        if i == len(srcind)-1:
            subtext = text[srcind[i]:-1]
        else:
            subtext = text[srcind[i]:srcind[i+1]]
            
        source.append(gettext('Source Name:','\n',subtext))
        pls.append(gettext('PLS Location:','\n',subtext))
        sourcetype.append(gettext('Source Type:','\n',subtext))
        sourceuse.append(gettext('Primary Use:','\n',subtext))
        win.append(gettext('Well ID Number:','(C',subtext))
        sourcecode.append(gettext('DEHN Source Code:','\n',subtext))
        wrnum.append(gettext('Water Right Numbers:','\n',subtext))
        system.append(systemname)
        systemid.append(sid)
        srcid = systid +'-'+str(i).zfill(2)
        sourceid.append(srcid)
        
        table = gettext(' Source Record (ACFT)\r\n','\r\n \r',subtext)
        table = re.sub('Master +Meter','MasterMeter',str(table))
        table = re.sub('Master +Met','MasterMeter',table)
        table = re.sub('Individual +Meters','IndividualMeters',table)
        table = re.sub('Measuring +Method','MeasuringMethod',table)
        table = tparser(table) 
        rv = table.split('\n')
        b = []
        for j in range(len(rv)):    
            if rv[j].count(',') > 15:
                rb = rv[j].split(',')
                rb.insert(15,'\n')
                b.append(','.join(rb))
            elif rv[j].count(',')==15:
                b.append(rv[j])  
            elif rv[j].count(',') < 15 and rv[j].count(',') > 4:
                b.append(rv[j] + ','*(15-rv[j].count(',')))
            else:
                pass
        rev = '\n'.join(b)
        try:
            sourcet[srcid] = pd.read_csv(StringIO(rev))
        except:
            pass
        
    usetable = gettext(' Annual Use Info (Acft) \r\n','\r\n \r',text)
    usetable = tparser(usetable)
    try:
        use[systid] = pd.read_csv(StringIO(usetable))
    except:
        pass
    
    conntable = gettext(' Annual Connection Info\r\n','\r\n\r\n ',text)
    conntable = tparser(conntable)
    try:
        conn[systid] = pd.read_csv(StringIO(conntable))
    except:
        pass


### Systems

In [27]:
sysdict = {'systype':systype, 'systemnum': sysnum, 'link':link, 'sysname':sysname, 'city':city, 
           'county':county, 'syscat':syscat, 'indust code':indcode, 'number of sources':numberofsources,
          'huc':huc, 'pwsid':pwsid,'deqcat':deqcat, 'systemid':system_id}

systems = pd.DataFrame(sysdict)

### Sources

In [28]:
sourcedict = {'source':source, 'system id': systemid, 'system':system, 'pls':pls, 'source type': sourcetype,
          'source use':sourceuse, 'win':win, 'wrnum':wrnum, 'DEHN source id':sourcecode, 'source id':sourceid}

sources = pd.DataFrame(sourcedict)

### Source Use

In [29]:
sourcetake = pd.concat(sourcet)
sourcetake.reset_index(inplace=True)
sourcetake.rename(columns={'level_0':'systemid'},inplace=True)
sourcetake.set_index(['systemid','Year'],inplace=True)
sourcetake.drop(['level_1','Measuring','MeasuringMethod','Unnamed: 15','Mea','Meth','Ann'],axis=1,inplace=True)

In [30]:
srctake = sourcetake.stack().to_frame()
srctake.rename(columns={'0':'Use (ac-ft)'},inplace=True)
srctake.reset_index(inplace=True)
srctake['Year'] = pd.to_numeric(srctake['Year'],errors='coerce')
srctake = srctake[(srctake['Year']<=datetime.today().year)&(srctake['Year']>=1000)]

In [31]:
srctake = srctake[srctake['level_2'].isin(['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])]
srctake['my'] = srctake[['Year','level_2']].apply(lambda x: pd.to_datetime(str(int(x[0]))+' '+str(x[1]), format='%Y %b'),1)
srctake.columns = ['sourceid','Year','Month','Use','datetime']
srctake['systemid'] = srctake['sourceid'].apply(lambda x: str(x)[0:9],1)

### System Use

In [32]:
systemuse = pd.concat(use)
systemuse.reset_index(inplace=True)
systemuse.rename(columns={'level_0':'systemid'},inplace=True)
systemuse.drop(['level_1','Tota','nan'],axis=1,inplace=True)
cols = ['Commercial','Domestic','Industrial','Institutnl','Other','Stock','Unmetered','Wholesale']
for col in cols:
    systemuse[col] = pd.to_numeric(systemuse[col], errors='coerce')
systemuse['Total'] = pd.to_numeric(systemuse['Total'])
systemuse['Total1'] = systemuse[cols].sum(axis=1)
systemuse['Year'] = pd.to_numeric(systemuse['Year'], errors='coerce')

In [34]:
systemuseData = systemuse[systemuse['Year']<2017]
systemuseData = systemuse[systemuse.Total < 100000]

# Append Existing Data

http://www.waterrights.utah.gov/cgi-bin/pubdump.exe?DBNAME=WELLDB&SECURITYKEY=wrt2012access<br/>
http://www.waterrights.utah.gov/cgi-bin/pubdump.exe?DBNAME=WRDB&SECURITYKEY=wrt2012access

In [8]:
wlactivity = pd.read_csv(route+'wlactivity.txt')

In [ ]:
replacements = {'""':'\""', 'temp':'bob', 'garbage':'nothing'}

with open(route+'wlcomments.txt') as infile, open(route+'wlcomments1.txt', 'w') as outfile:
    for line in infile:
        for src, target in replacements.iteritems():
            line = line.replace(src, target)
        outfile.write(line)

In [4]:
wlcomments = pd.read_csv(route+'wlcomments.txt',delimiter='\t',error_bad_lines=False)

In [5]:
cols = ['win','finishedDiameter','drillingMethod','totalDepth','finishedDepth','screenDepth','geologicLog']
wlwellfeatures = pd.read_csv(route+'wlwellfeatures.txt',header = None, 
                             names = cols, delimiter='\t',error_bad_lines=False)

In [69]:
cols = ['recordId', 'win', 'wlDate', 'wlTime', 'wlDepth', 'wlStatus', 'wlMethod']       
wlWaterLevel = pd.read_csv(route+'wlWaterLevel.txt',header = None, names = cols, delimiter='\t',error_bad_lines=False,)

def timefix(x):
    if x[1]=='' or str(x[1]) == np.nan or str(x[1]) =='nan':
        x[1]='00:00:00'
    return pd.to_datetime(str(x[0])+' '+str(x[1]), format='%Y%m%d %H:%M:%S',errors='coerce',exact='false')


wlWaterLevel['datetime'] = wlWaterLevel[['wlDate','wlTime']].apply(lambda x: timefix(x),1)

In [70]:
wlWaterLevel

,recordId,win,wlDate,wlTime,wlDepth,wlStatus,wlMethod,datetime
0,131180,3440,20140923,10:41:38,58.00000,toc,sounder,2014-09-23 10:41:38
1,131181,436593,20140923,10:46:43,7.00000,toc,elec sounder,2014-09-23 10:46:43
2,131182,437767,20140924,15:29:56,0.00000,flowing,NaN,2014-09-24 15:29:56
3,131183,437898,20140924,15:46:23,235.00000,ground,probe,2014-09-24 15:46:23
4,131184,437905,20140925,11:10:15,120.00000,ground,sounder,2014-09-25 11:10:15
5,131185,437930,20140925,11:12:10,0.00000,flowing,NaN,2014-09-25 11:12:10
6,131187,437917,20140929,14:57:42,270.00000,toc,water tape,2014-09-29 14:57:42
7,131188,437841,20140930,13:54:00,170.00000,ground,TAPE,2014-09-30 13:54:00
8,131189,437988,20141001,13:31:50,38.50000,topofeight,wlm,2014-10-01 13:31:50
9,131190,437974,20141001,13:33:08,42.00000,topofeight,wlm,2014-10-01 13:33:08


The following converts the harvested data to water rights text format

In [100]:
quer = "SELECT * FROM water_rights_data.waterlevels"
wls = pd.read_sql_query(sql=quer,con=engine)
wls['Date'] = wls['Date'].apply(lambda x: pd.to_datetime(x, errors='coerce'), 1)
wls = wls[(wls['Date'].notnull()) & (wls['Date'] >= pd.datetime(1900,1,1))]
wls['wlDate'] = wls['Date'].apply(lambda x: "{:%Y%m%d}".format(x),1)
wls.columns = ['win','datetime','wlDepth','wlStatus', u'wlDate']
wlWL = pd.concat([wls,wlWaterLevel],axis=0)
wlWL.drop(['datetime'],axis=1,inplace=True)
wlWL['wlStatus']=wlWL['wlStatus'].apply(lambda x: str(x).title(),1)
wlWL.sort_values(by=['win','recordId'], inplace=True)
wlWL.drop_duplicates(subset=['win','wlDate','wlDepth'],keep='first',inplace=True)
wlWL.to_csv(route+'wlWaterLevel_v1.txt', sep='\t')

The following combines existing water rights data with the harvested data for upload to arcgis.

In [137]:
quer = "SELECT * FROM water_rights_data.waterlevels"
wls = pd.read_sql_query(sql=quer,con=engine)
wls['Date'] = wls['Date'].apply(lambda x: pd.to_datetime(x, errors='coerce'), 1)
wls = wls[(wls['Date'].notnull())]
wls['Level'] = wls['Level'].apply(lambda x: round(pd.to_numeric(x, errors='coerce'),2),1)
wrData = wlWaterLevel.drop(['recordId','wlTime','wlMethod','wlDate'],axis=1)
wrData.columns = ['WIN','Level','Method','Date']
wrData['Level'] = wrData['Level'].apply(lambda x: round(pd.to_numeric(x, errors='coerce'),2),1)
waterlevels = pd.concat([wls,wrData],axis=0)
waterlevels['Method'] = waterlevels['Method'].apply(lambda x: str(x).title(),1)

waterlevels.sort_values(by=['WIN','Date'], inplace=True)
waterlevels.drop_duplicates(subset=['WIN','Date'],keep='last',inplace=True)
waterlevels = waterlevels[waterlevels['Date'].notnull()]
waterlevels.columns

Index([u'Date', u'Level', u'Method', u'WIN'], dtype='object')

In [138]:
waterlevels.columns = ['Date', 'Level', 'wlMethod', 'WIN']

# Upload Data

### System Connections

In [36]:
connections = pd.concat(conn)
connections.reset_index(inplace=True)
connections.rename(columns={'level_0':'systemid'},inplace=True)
connections.drop(['level_1','nan'],axis=1,inplace=True)
connections = connections[(connections['Year']<datetime.today().year)&(connections['Year']>1830)]
connections = connections[connections.Total < 100000]

## MySQL

This must be done after scraping or you will throw a host error.

In [8]:
engineroute = "H:/Google Drive/WORK/Groundwater Chemistry"
#engineroute = "C:/Users/Brooke/Downloads/"
sys.path.append(engineroute)
import enginegetter

In [9]:
engine = enginegetter.getEngine()

### Systems and Sources

These are the systems made up of the sources.  They are often cities or water agencies.

In [39]:
systems.to_sql(con=engine, name='systems', if_exists='replace', flavor='mysql',index=False)

These are the water use sources.

In [40]:
sources.to_sql(con=engine, name='sources', if_exists='replace', flavor='mysql', index=False)

Depicts the amount of water use by each source in ac-ft/mo

In [41]:
srctake.to_sql(con=engine, name='sourceuse', if_exists='replace', flavor='mysql',chunksize=10000)

Depicts the amount of water use by each system in ac-ft/yr

In [42]:
systemuseData.to_sql(con=engine, name='systemuse', if_exists='replace', flavor='mysql',chunksize=10000)

Depicts number of connections in system

In [43]:
connections.to_sql(con=engine, name='systemconnections', if_exists='replace', flavor='mysql')

### Well Data

In [44]:
waterlevels.to_sql(con=engine, name='waterlevels', if_exists='replace', flavor='mysql',index=False)

In [45]:
borehole.to_sql(con=engine, name='borehole', if_exists='replace', flavor='mysql',index=False)

In [46]:
driller.to_sql(con=engine, name='driller', if_exists='replace', flavor='mysql',index=False)

In [47]:
lithlog.to_sql(con=engine, name='lithlog', if_exists='replace', flavor='mysql',index=False, chunksize=1000)

In [48]:
construction.to_sql(con=engine, name='construction', if_exists='replace', flavor='mysql',index=False)

In [49]:
screendf.to_sql(con=engine, name='wellscreens', if_exists='replace', flavor='mysql',index=False)

In [50]:
wellTrans.to_sql(con=engine, name='pumpingtests', if_exists='replace', flavor='mysql',index=False)

## ArcPy

In [98]:
'''
Locate ArcPy and add it to the path
Created on 13 Feb 2015
@author: Jamesramm
https://github.com/JamesRamm/archook/blob/master/archook.py
'''
import _winreg
import sys
from os import path
def locate_arcgis():
  '''
  Find the path to the ArcGIS Desktop installation.
  Keys to check:
  HLKM/SOFTWARE/ESRI/ArcGIS 'RealVersion' - will give the version, then we can use
  that to go to
  HKLM/SOFTWARE/ESRI/DesktopXX.X 'InstallDir'. Where XX.X is the version
  We may need to check HKLM/SOFTWARE/Wow6432Node/ESRI instead
  '''
  try:
    key = _winreg.OpenKey(_winreg.HKEY_LOCAL_MACHINE,
                          'SOFTWARE\\Wow6432Node\\ESRI\\ArcGIS', 0)

    version = _winreg.QueryValueEx(key, "RealVersion")[0][:4]

    key_string = "SOFTWARE\\Wow6432Node\\ESRI\\Desktop{0}".format(version)
    desktop_key = _winreg.OpenKey(_winreg.HKEY_LOCAL_MACHINE,
                                  key_string, 0)

    install_dir = _winreg.QueryValueEx(desktop_key, "InstallDir")[0]
    return install_dir
  except WindowsError:
    raise ImportError("Could not locate the ArcGIS directory on this machine")

def get_arcpy():  
  '''
  Allows arcpy to imported on 'unmanaged' python installations (i.e. python installations
  arcgis is not aware of).
  Gets the location of arcpy and related libs and adds it to sys.path
  '''
  install_dir = locate_arcgis()  
  arcpy = path.join(install_dir, "arcpy")
  # Check we have the arcpy directory.
  if not path.exists(arcpy):
    raise ImportError("Could not find arcpy directory in {0}".format(install_dir))

  # First check if we have a bin64 directory - this exists when arcgis is 64bit
  bin_dir = path.join(install_dir, "bin64")
  if not path.exists(bin_dir):
    # Fall back to regular 'bin' dir otherwise.
    bin_dir = path.join(install_dir, "bin")

  scripts = path.join(install_dir, "ArcToolbox", "Scripts")  
  sys.path.extend([arcpy, bin_dir, scripts])

In [99]:
get_arcpy()
import arcpy

In [123]:
fileloc = r'H:/GIS/WR_DATA.gdb/'

def df2gdb(df,fileloc,name):
    x = np.array(np.rec.fromrecords(df.values))
    names = df.dtypes.index.tolist()
    names = [str(i) for i in names]
    x.dtype.names = tuple(names)
    arcpy.da.NumPyArrayToTable(x, fileloc+name)

def df2csv2gdb(df,fileloc,csvloc,name,ind=False):
    df.to_csv(csvloc,index=ind)
    arcpy.TableToTable_conversion(csvloc, fileloc, name)


### Systems and Sources

In [ ]:
df2gdb(systems,fileloc,'systems')

In [ ]:
df2gdb(sources,fileloc,'sources')

In [ ]:
df2gdb(systemuseData,fileloc,'systemuse')

### Well Data

In [139]:
df2csv2gdb(waterlevels,fileloc,r'H:/GIS/waterlevels.csv','waterlevels')

In [ ]:
df2gdb(borehole,fileloc,'borehole')

In [ ]:
df2gdb(driller,fileloc,'driller')

In [ ]:
df2csv2gdb(lithlog,fileloc,r'C:/GIS/lithlogs.csv','lithlogs')

In [ ]:
df2gdb(construction,fileloc,'construction')

In [ ]:
df2gdb(screendf,fileloc,'screen')

In [ ]:
df2gdb(wellTrans,fileloc,'spCapTrans')

In [ ]:
df2gdb(connections,fileloc,'connections')

# Plot and Analyze Data

In [ ]:
systemuseData.groupby('Year')[['Total','Domestic','Industrial','Commercial']].sum().plot()
plt.xlim(1980,2020)
plt.ylabel('Use (ac-ft)')

In [ ]:
systemuseData.groupby('Year')['Total'].sum().plot()
plt.xlim(1980,2020)
plt.ylabel('Use (ac-ft)')

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('C:\\PROJECTS\\WR_DATA\\' + 'systems_and_sources.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
systems.to_excel(writer, sheet_name='systems')
sources.to_excel(writer, sheet_name='sources')
srctakeData.to_excel(writer, sheet_name='sourcetake')
systemuseData.to_excel(writer, sheet_name='system_use')
# Close the Pandas Excel writer and output the Excel file.
writer.save()